# Setup

In [ ]:
import pandapower.networks as pnet
import numpy as np

In [ ]:
import sys
sys.path.insert(1, '..')

from src.simulation.noise import add_noise_in_cartesian_coordinates
from src.models.regression import ComplexRegression, ComplexLasso
from src.simulation.load_profile import generate_gaussian_load
from src.simulation.network import add_load_power_control, make_y_bus
from src.simulation.simulation import run_simulation, get_current_and_voltage
from src.identification.error_metrics import error_metrics, fro_error

# Network simulation
To start, we simulate Gaussian and independent fluctuations for power demands of loads, close to their referece values.
We collect current $I_i$ and voltage $V_i$ at each node for $n$ time steps, $i = 1...n$.
By first principles, we know that $I_i = YV_i$, where Y is the bus admittance matrix.

In [ ]:
net = pnet.case33bw()
nodes = net.bus.shape[0]
steps = 30000
load_cv = 0.02
current_accuracy = 0.0001
voltage_accuracy = 0.0001

In [ ]:
load_p, load_q = generate_gaussian_load(net.load.p_mw, net.load.q_mvar, load_cv, steps)
controlled_net = add_load_power_control(net, load_p, load_q)
sim_result = run_simulation(controlled_net, verbose=False)
y_bus = make_y_bus(controlled_net)
voltage, current = get_current_and_voltage(sim_result, y_bus)
noisy_voltage, noisy_current = add_noise_in_cartesian_coordinates(current, voltage, current_accuracy, voltage_accuracy)

# OLS Identification
We start with a classical OLS regression. We define:
\begin{align}
I &= [I_1^T, ..., I_n^T]^T \\
V &= [V_1^T, ..., V_n^T]^T
\end{align}
Then, we have:
\begin{equation}
Y_{ols} = (V^HV)^{-1}V^HI
\end{equation}

In [ ]:
y_ols = ComplexRegression().fit(noisy_voltage, noisy_current).fitted_beta
error_metrics(y_bus, y_ols)

# Lasso Identification

In order to apply the complex LASSO algirithm, we first need to perform a vectorization of our model, as in (Ardakanian, 2019). Let $\mathbb{I}_n$ represent the n-by-n identity matrix. We have: 
\begin{equation}
\text{vect}(I) = \text{vect}(VY) = \left( \mathbb{I}_n \otimes V \right) \text{vect} (Y).
\end{equation}

Thus, the optimization problem we need to solve is:
\begin{equation}
\hat x = \arg \min_x \left \Vert b - Ax \right \Vert^2_2 + \lambda \left \Vert x \right \Vert_1.
\end{equation}

where:
\begin{align}
A&= \text{vect}{(V^T)} \otimes \mathbb{I}_n, \\
b &= \text{vect} (I), \\
x &= \text{vect} (Y),
\end{align}

In [ ]:
b = np.ravel(noisy_current, 'F')
a = np.kron(np.eye(nodes), noisy_voltage)

Then, we can apply the r-LASSO, defined in (Maleki, 2013), following what done by (Ardakanian, 2019) - see https://github.com/sustainable-computing/Distribution-Grid-Identification/blob/master/standardLasso.m.

In order to solve the problem, we frame it as an uncontrained convex optimization problem, using the Python cvxpy library: https://github.com/cvxgrp/cvxpy.

In [ ]:
#lasso_res = ComplexLasso(verbose=False, lambdas=np.logspace(-10, 1, 20)).fit(a, b)
#best_lasso_res = lasso_res.get_best_by(lambda y: fro_error(y.reshape(nodes, nodes), y_bus))

In [ ]:
# best_lasso_res.lambda_value

In [ ]:
# error_metrics(y_bus, best_lasso_res.fitted_beta.reshape(nodes, nodes))

# TLS Identification

In [ ]:
%%time
u, s, vh = np.linalg.svd(np.block([noisy_voltage, noisy_current]))
v = vh.conj().T
v_xy = v[:nodes, nodes:]
v_yy = v[nodes:, nodes:]
y_tls = - v_xy @ np.linalg.inv(v_yy)

In [ ]:
error_metrics(y_bus, y_tls)